# STXGB Model

This notebook contains the code for developing STXGB models. STXGB has three variants (STXGB-FB, STXGB-SG, and STXGB-SGR) and first we define the features that we have used in each variant. Then for each prediction horizon, we train a separate model using XGBoost algorithm.

In [ ]:
import pandas as pd
import numpy as np 
import sklearn
import geopandas as gpd
from scipy.stats import norm
import time
import os

In [ ]:
import xgboost

In [ ]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set output directory
# You can change it if you want to
output_dir = './output/'

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## 1- Loading Data
The `all_features_v1.csv` contains all of the features and target variables that we have used in different variants of STXGB model and for 1- to 4-week prediction horizons. 


This file is published publicly alongside the code, so you can download the csv file and run STXGB models for yourself.

In [ ]:
# here we load data from Zenodo URL

df_url = 'https://zenodo.org/record/5533027/files/all_features_v1_0.csv?download=1'
covid_df = pd.read_csv(df_url,index_col=0, dtype={'GEOID':str})

In [ ]:
covid_df.head()

In [ ]:
#Check the number of weeks for which we have data
covid_df.shape[0]/3103 

In [ ]:
covid_df.isnull().sum().sum()

#### Load a base geojson file 

This file contains county FIPS and is used to store model outputs is a georeferenced format.

In [ ]:
url='https://drive.google.com/file/d/1MVyLzzHl3hzno4o1rLZtI0peqIi23zsr/view?usp=sharing'
url_counties='https://drive.google.com/uc?id=' + url.split('/')[-2]
counties_shp = gpd.read_file(url_counties)

__Now, we have to define feature names for each of the models. Please refer to the methods section of the article to read more about these features.__

### 1.1. The list of features in the base model (base features)

In [ ]:
temp_cols = [col for col in covid_df.columns if 'TEMP' in col]

In [ ]:
socio_cols = ['POP_DENSITY',
'PCT_MALE',
'PCT_65_OVE',
'PCT_BLACK',
'PCT_HISPAN', 
'PCT_AMIND',
'PCT_RURAL',
'PCT_COL_DE' ,
'PCT_TRUMP_',
'MED_HOS_IN']

In [ ]:
inc_cols = [col for col in covid_df.columns if 'DELTA_INC' in col]
inc_cols.pop(0)

In [ ]:
base_features = socio_cols + temp_cols + inc_cols + ['LOG_MEAN_INC_RATE_T_4']

### 1.2. The list of features in the STXGB-FB model

In [ ]:
spc_cols = [col for col in covid_df.columns if 'DELTA_SPC_T' in col]

In [ ]:
rel_cols = [col for col in covid_df.columns if 'REL_' in col]
rel_cols_non_delta = [col for col in rel_cols if 'DELTA' in col]
rel_cols = list(set(rel_cols)^set(rel_cols_non_delta))

In [ ]:
ratio_cols = [col for col in covid_df.columns if 'RATIO_' in col]
ratio_cols_non_delta = [col for col in ratio_cols if 'DELTA' in col]
ratio_cols = list(set(ratio_cols)^set(ratio_cols_non_delta))

In [ ]:
facebook_features = socio_cols + temp_cols + rel_cols + ratio_cols + spc_cols  + inc_cols  

In [ ]:
facebook_features.extend(('LOG_MEAN_INC_RATE_T_4', 'MEAN_SPC_T_4'))

### 1.3. The list of features in the STXGB-SG model

In [ ]:
fpc_cols = [col for col in covid_df.columns if 'DELTA_FPC_T' in col]

In [ ]:
pct_home_cols = [col for col in covid_df.columns if 'completely_home_' in col]
pct_home_cols_non_base = [col for col in pct_home_cols if 'baselined' in col]
pct_home_cols = list(set(pct_home_cols)^set(pct_home_cols_non_base))

In [ ]:
dist_traveled_cols = [col for col in covid_df.columns if 'distance_traveled_' in col]
dist_traveled_cols_non_current = [col for col in dist_traveled_cols if 'current' in col]
dist_traveled_cols = list(set(dist_traveled_cols)^set(dist_traveled_cols_non_current))

In [ ]:
safegraph_features = socio_cols + temp_cols + pct_home_cols + dist_traveled_cols + \
                    fpc_cols + inc_cols

In [ ]:
safegraph_features.extend(('LOG_MEAN_INC_RATE_T_4','MEAN_FPC_T_4'))

### 1.4. The list of features in the STXGB-SGR model

In [ ]:
baselined_cols = [col for col in covid_df.columns if 'baselined_' in col]

In [ ]:
slope_cols = [col for col in covid_df.columns if 'slope_' in col]

In [ ]:
safegraph_full_features = safegraph_features + baselined_cols + slope_cols

In [ ]:
safegraph_full_features = set(safegraph_full_features)

In [ ]:
 features_to_remove=['pct_completely_home_device_count_current_T_1',
 'pct_completely_home_device_count_current_T_2',
 'pct_completely_home_device_count_current_T_3',
 'pct_completely_home_device_count_current_T_4',
 'pct_delivery_behavior_devices_baselined_T_1',
 'pct_delivery_behavior_devices_baselined_T_2',
 'pct_delivery_behavior_devices_baselined_T_3',
 'pct_delivery_behavior_devices_baselined_T_4',
 'pct_delivery_behavior_devices_slope_T_1',
 'pct_delivery_behavior_devices_slope_T_2',
 'pct_delivery_behavior_devices_slope_T_3',
 'pct_delivery_behavior_devices_slope_T_4',
 'pct_part_time_work_behavior_devices_baselined_T_1',
 'pct_part_time_work_behavior_devices_baselined_T_2',
 'pct_part_time_work_behavior_devices_baselined_T_3',
 'pct_part_time_work_behavior_devices_baselined_T_4',
 'pct_part_time_work_behavior_devices_slope_T_1',
 'pct_part_time_work_behavior_devices_slope_T_2',
 'pct_part_time_work_behavior_devices_slope_T_3',
 'pct_part_time_work_behavior_devices_slope_T_4']

In [ ]:
safegraph_full_features = list(safegraph_full_features)

In [ ]:
safegraph_full_features = [i for i in safegraph_full_features if i not in features_to_remove]

### 1.5. Removing inf values from the dataframe

In [ ]:
covid_df = covid_df.replace([np.inf, -np.inf], np.NaN)

In [ ]:
covid_df.isna().sum().sum()

In [ ]:
na_cols = covid_df.columns[covid_df.isna().any()].tolist()

In [ ]:
for col in na_cols:
    covid_df[col] = covid_df.groupby(['date_start_period','STATEFP'])[col].transform(lambda x: x.fillna(x.mean()))

In [ ]:
covid_df.isna().sum().sum()

## 2. Set training and testing size

The dataset is initially divided into a 34-week subset for training and a 1-week subset for testing.

At each time step, the size of training weeks increases by 1 and the test week has a 1-week shift towards the end of November

In [ ]:
training_size = 30 # week
testing_size = 1 # week
num_counties = 3103
time_steps = 14

## 3. Training STXGB for one-week (7-day) prediction horizon

In [ ]:
counties_xgb = counties_shp.copy()

In [ ]:
train_r2_xgb = dict()
train_rmse_xgb = dict()
train_mae_xgb = dict()
test_rmse_xgb = dict()
test_mae_xgb = dict()
tuned_params_xgb = dict()



models=['base', 'safegraph', 'facebook', 'safegraph_full']
features = [base_features, safegraph_features, facebook_features, safegraph_full_features]

# Setting Hyperparameters. Please refer to the SI for more information
xgb_params = dict(learning_rate=np.arange(0.05,0.3,0.05), 
                     n_estimators=np.arange(100,1000,100), 
                     gamma = np.arange(1,10,1),
                     subsample = np.arange(0.1,0.5,0.05),
                     max_depth=[int(i) for i in np.arange(1,10,1)]) 



for i in range(time_steps):
    
    training_df = covid_df.iloc[:(i+training_size)*num_counties,:]
    testing_df = covid_df.iloc[(i+training_size)*num_counties:(i+training_size+testing_size)*num_counties,:]
    
    for model,feature in zip(models, features):
        
        time_start = time.time()
        
        X_train = training_df[feature]
        y_train = training_df['LOG_DELTA_INC_RATE_T']
        X_test = testing_df[feature]
        y_test = testing_df['LOG_DELTA_INC_RATE_T'] 

        #scaling X
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)


        #inititalization
        xgb_model = xgboost.XGBRegressor(booster='gbtree', seed=42, verbosity=0) 
        
        #cross validation
        xgb_cv = RandomizedSearchCV(xgb_model, xgb_params, random_state=21, 
                                    scoring='neg_root_mean_squared_error', n_jobs=-1)
        xgb_optimized = xgb_cv.fit(X_train, y_train)
        best_xgb = xgb_optimized.best_estimator_
        tuned_params_xgb[model, i] = xgb_optimized.best_params_

        # model evaluation for training set
        r2_train_xgb = round(best_xgb.score(X_train, y_train),2)
        train_r2_xgb[model, i] = r2_train_xgb

        y_train_predicted_xgb = best_xgb.predict(X_train)
        rmse_train_xgb = (np.sqrt(mean_squared_error(y_train, y_train_predicted_xgb)))
        train_rmse_xgb[model, i] = rmse_train_xgb
        train_mae_xgb[model, i] =  mean_absolute_error(y_train, y_train_predicted_xgb)

        # model evaluation for test set
        y_test_predicted_xgb = best_xgb.predict(X_test)
        rmse_test_xgb = (np.sqrt(mean_squared_error(y_test, y_test_predicted_xgb)))
        test_rmse_xgb[model, i] = rmse_test_xgb
        test_mae_xgb[model, i] = mean_absolute_error(y_test, y_test_predicted_xgb)
        
        #feature importance
        
        xgb_importance = pd.concat([pd.DataFrame(feature, columns={'variable'}),
                                    pd.DataFrame(np.transpose(best_xgb.feature_importances_), columns={'Importance'})],
                                   axis = 1) 
        xgb_importance.sort_values(by='Importance', ascending=False, inplace=True)
        xgb_importance.to_csv(output_dir + 'XGB_importance_' + model + '_' + str(i) + '.csv')
        
        # add true values and predictions to the county data frame
        col_suffix = model +'_' + str(i)
        
        testing_df.loc[:,'y_test_'+ col_suffix] = y_test
        testing_df.loc[:,'y_predicted_'+ col_suffix] = y_test_predicted_xgb
        
        testing_df['delta_inc_test_'+ col_suffix] = np.exp(testing_df['y_test_'+ col_suffix]) - 1
        testing_df['delta_inc_pred_'+ col_suffix] = np.exp(testing_df['y_predicted_'+ col_suffix]) - 1
        
        testing_df['delta_case_test_'+ col_suffix] = (testing_df['delta_inc_test_'+ col_suffix] * 
                                                        testing_df['POPULATION']) / 10000
        
        testing_df['delta_case_pred_'+ col_suffix] = (testing_df['delta_inc_pred_'+ col_suffix] * 
                                                        testing_df['POPULATION']) / 10000
        
        testing_df['error_y_'+ col_suffix] = testing_df['y_test_'+ col_suffix] - testing_df['y_predicted_'+ col_suffix]
        
        testing_df['error_delta_inc_'+ col_suffix] = testing_df['delta_inc_test_'+ col_suffix] - \
                                                        testing_df['delta_inc_pred_'+ col_suffix]
        
        testing_df['error_delta_case_'+ col_suffix] = testing_df['delta_case_test_'+ col_suffix] - \
                                                        testing_df['delta_case_pred_'+ col_suffix]
        
        test_cols = ['GEOID',  
                      'y_test_'+ col_suffix, 'y_predicted_'+ col_suffix, 
                      'delta_inc_test_'+ col_suffix,  'delta_inc_pred_'+ col_suffix,
                      'delta_case_test_'+ col_suffix, 'delta_case_pred_'+ col_suffix,
                      'error_y_'+ col_suffix, 'error_delta_inc_'+ col_suffix, 'error_delta_case_'+ col_suffix]
        
        
        counties_xgb = counties_xgb.merge(testing_df[test_cols], how='left', on='GEOID')
        
        time_end = time.time()
        print('Training Model {} in time step {} finished in {} seconds!'.format(model, i, round(time_end-time_start, 2)))

In [ ]:
# save the output for performance analysis
counties_xgb.to_file(output_dir + 'STXGB_1week_pred.geojson', driver='GeoJSON')

## 4. Training STXGB for two-week (14-day) prediction horizon

In [ ]:
counties_xgb_14 = counties_shp.copy()

In [ ]:
train_r2_xgb_14 = dict()
train_rmse_xgb_14 = dict()
train_mae_xgb_14 = dict()
test_rmse_xgb_14 = dict()
test_mae_xgb_14 = dict()
tuned_params_xgb_14 = dict()



models=['base', 'safegraph', 'facebook', 'safegraph_full']
features = [base_features, safegraph_features, facebook_features, safegraph_full_features]

# Setting Hyperparameters. Please refer to the SI for more information
xgb_params = dict(learning_rate=np.arange(0.05,0.3,0.05), 
                     n_estimators=np.arange(100,1000,100), 
                     gamma = np.arange(1,10,1),
                     subsample = np.arange(0.1,0.5,0.05),
                     max_depth=[int(i) for i in np.arange(1,10,1)]) 



for i in range(time_steps):
    
    training_df = covid_df.iloc[:(i+training_size)*num_counties,:]
    testing_df = covid_df.iloc[(i+training_size)*num_counties:(i+training_size+testing_size)*num_counties,:]
    
    
    for model,feature in zip(models, features):
        
        time_start = time.time()
        
        # in the 2-week prediction model, the target variable is LOG_DELTA_INC_RATE_T_14
        X_train = training_df[feature]
        y_train = training_df['LOG_DELTA_INC_RATE_T_14']
        X_test = testing_df[feature]
        y_test = testing_df['LOG_DELTA_INC_RATE_T_14'] 

        #scaling X
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)


        #inititalization
        xgb_model = xgboost.XGBRegressor(booster='gbtree', seed=42, verbosity=0) 
        
        #cross validation
        xgb_cv = RandomizedSearchCV(xgb_model, xgb_params, random_state=21, 
                                    scoring='neg_root_mean_squared_error', n_jobs=-1)
        
        xgb_optimized = xgb_cv.fit(X_train, y_train)
        best_xgb = xgb_optimized.best_estimator_
        tuned_params_xgb_14[model, i] = xgb_optimized.best_params_

        # model evaluation for training set
        r2_train_xgb = round(best_xgb.score(X_train, y_train),2)
        train_r2_xgb_14[model, i] = r2_train_xgb

        y_train_predicted_xgb = best_xgb.predict(X_train)
        rmse_train_xgb = (np.sqrt(mean_squared_error(y_train, y_train_predicted_xgb)))
        train_rmse_xgb_14[model, i] = rmse_train_xgb
        train_mae_xgb_14[model, i] =  mean_absolute_error(y_train, y_train_predicted_xgb)

        # model evaluation for test set
        y_test_predicted_xgb = best_xgb.predict(X_test)
        rmse_test_xgb = (np.sqrt(mean_squared_error(y_test, y_test_predicted_xgb)))
        test_rmse_xgb_14[model, i] = rmse_test_xgb
        test_mae_xgb_14[model, i] = mean_absolute_error(y_test, y_test_predicted_xgb)
        
        #feature importance
        
        xgb_importance = pd.concat([pd.DataFrame(feature, columns={'variable'}),
                                    pd.DataFrame(np.transpose(best_xgb.feature_importances_), columns={'Importance'})],
                                   axis = 1) 
        xgb_importance.sort_values(by='Importance', ascending=False, inplace=True)
        xgb_importance.to_csv(output_dir + 'XGB_importance_14_' + model + '_' + str(i) + '.csv')
        
        # add true values and predictions to the county data frame
        col_suffix = model +'_' + str(i)
        
        testing_df.loc[:,'y_test_'+ col_suffix] = y_test
        testing_df.loc[:,'y_predicted_'+ col_suffix] = y_test_predicted_xgb
        
        testing_df['delta_inc_test_'+ col_suffix] = np.exp(testing_df['y_test_'+ col_suffix]) - 1
        testing_df['delta_inc_pred_'+ col_suffix] = np.exp(testing_df['y_predicted_'+ col_suffix]) - 1
        
        testing_df['delta_case_test_'+ col_suffix] = (testing_df['delta_inc_test_'+ col_suffix] * 
                                                      testing_df['POPULATION']) / 10000
        
        testing_df['delta_case_pred_'+ col_suffix] = (testing_df['delta_inc_pred_'+ col_suffix] * 
                                                      testing_df['POPULATION']) / 10000
        
        testing_df['error_y_'+ col_suffix] = testing_df['y_test_'+ col_suffix] - testing_df['y_predicted_'+ col_suffix]
        
        testing_df['error_delta_inc_'+ col_suffix] = testing_df['delta_inc_test_'+ col_suffix] - \
                                                        testing_df['delta_inc_pred_'+ col_suffix]
        
        testing_df['error_delta_case_'+ col_suffix] = testing_df['delta_case_test_'+ col_suffix] - \
                                                        testing_df['delta_case_pred_'+ col_suffix]
        
        test_cols = ['GEOID',  
                      'y_test_'+ col_suffix, 'y_predicted_'+ col_suffix, 
                      'delta_inc_test_'+ col_suffix,  'delta_inc_pred_'+ col_suffix,
                      'delta_case_test_'+ col_suffix, 'delta_case_pred_'+ col_suffix,
                      'error_y_'+ col_suffix, 'error_delta_inc_'+ col_suffix, 'error_delta_case_'+ col_suffix]
        
        
        counties_xgb_14 = counties_xgb_14.merge(testing_df[test_cols], how='left', on='GEOID')
        
        time_end = time.time()
        print('Training Model {} in time step {} finished in {} seconds!'.format(model, i, round(time_end-time_start, 2)))
    

In [ ]:
# save the output for performance analysis
counties_xgb_14.to_file(output_dir + 'STXGB_2week_pred.geojson', driver='GeoJSON')

## 5. Training STXGB for three-week (21-day) prediction horizon

In [ ]:
train_r2_xgb_21 = dict()
train_rmse_xgb_21 = dict()
train_mae_xgb_21 = dict()
test_rmse_xgb_21 = dict()
test_mae_xgb_21 = dict()
tuned_params_xgb_21 = dict()



models=['base', 'safegraph', 'facebook', 'safegraph_full']
features = [base_features, safegraph_features, facebook_features, safegraph_full_features]

# Setting Hyperparameters. Please refer to the SI for more information
xgb_params = dict(learning_rate=np.arange(0.05,0.3,0.05), 
                     n_estimators=np.arange(100,1000,100), 
                     gamma = np.arange(1,10,1),
                     subsample = np.arange(0.1,0.5,0.05),
                     max_depth=[int(i) for i in np.arange(1,10,1)]) 



for i in range(time_steps):
    time_start = time.time()
    
    training_df = covid_df.iloc[:(i+training_size)*num_counties,:]
    testing_df = covid_df.iloc[(i+training_size)*num_counties:(i+training_size+testing_size)*num_counties,:]
    
    
    for model,feature in zip(models, features):
        
        # in the 3-week prediction model, the target variable is LOG_DELTA_INC_RATE_T_21
        X_train = training_df[feature]
        y_train = training_df['LOG_DELTA_INC_RATE_T_21']
        X_test = testing_df[feature]
        y_test = testing_df['LOG_DELTA_INC_RATE_T_21'] 

        #scaling X
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)


        #inititalization
        xgb_model = xgboost.XGBRegressor(booster='gbtree', seed=42, verbosity=0) 
        
        #cross validation
        xgb_cv = RandomizedSearchCV(xgb_model, xgb_params, random_state=21, 
                                    scoring='neg_root_mean_squared_error', n_jobs=-1)
        xgb_optimized = xgb_cv.fit(X_train, y_train)
        best_xgb = xgb_optimized.best_estimator_
        tuned_params_xgb_21[model, i] = xgb_optimized.best_params_

        # model evaluation for training set
        r2_train_xgb = round(best_xgb.score(X_train, y_train),2)
        train_r2_xgb_21[model, i] = r2_train_xgb

        y_train_predicted_xgb = best_xgb.predict(X_train)
        rmse_train_xgb = (np.sqrt(mean_squared_error(y_train, y_train_predicted_xgb)))
        train_rmse_xgb_21[model, i] = rmse_train_xgb
        train_mae_xgb_21[model, i] =  mean_absolute_error(y_train, y_train_predicted_xgb)

        # model evaluation for test set
        y_test_predicted_xgb = best_xgb.predict(X_test)
        rmse_test_xgb = (np.sqrt(mean_squared_error(y_test, y_test_predicted_xgb)))
        test_rmse_xgb_21[model, i] = rmse_test_xgb
        test_mae_xgb_21[model, i] = mean_absolute_error(y_test, y_test_predicted_xgb)
        
        #feature importance
        
        xgb_importance = pd.concat([pd.DataFrame(feature, columns={'variable'}),
                                    pd.DataFrame(np.transpose(best_xgb.feature_importances_), columns={'Importance'})],
                                   axis = 1) 
        xgb_importance.sort_values(by='Importance', ascending=False, inplace=True)
        xgb_importance.to_csv(output_dir + 'XGB_importance_21_' + model + '_' + str(i) + '.csv')
        
        # # add labels and predictions to a county data frame
        col_suffix = model +'_' + str(i)
        
        testing_df.loc[:,'y_test_'+ col_suffix] = y_test
        testing_df.loc[:,'y_predicted_'+ col_suffix] = y_test_predicted_xgb
        
        testing_df['delta_inc_test_'+ col_suffix] = np.exp(testing_df['y_test_'+ col_suffix]) - 1
        testing_df['delta_inc_pred_'+ col_suffix] = np.exp(testing_df['y_predicted_'+ col_suffix]) - 1
        
        testing_df['delta_case_test_'+ col_suffix] = (testing_df['delta_inc_test_'+ col_suffix] * 
                                                    testing_df['POPULATION']) / 10000
        
        testing_df['delta_case_pred_'+ col_suffix] = (testing_df['delta_inc_pred_'+ col_suffix] * 
                                                      testing_df['POPULATION']) / 10000
        
        testing_df['error_y_'+ col_suffix] = testing_df['y_test_'+ col_suffix] - testing_df['y_predicted_'+ col_suffix]
        
        testing_df['error_delta_inc_'+ col_suffix] = testing_df['delta_inc_test_'+ col_suffix] - \
                                                        testing_df['delta_inc_pred_'+ col_suffix]
        
        testing_df['error_delta_case_'+ col_suffix] = testing_df['delta_case_test_'+ col_suffix] - \
                                                        testing_df['delta_case_pred_'+ col_suffix]
        
        test_cols = ['GEOID',  
                      'y_test_'+ col_suffix, 'y_predicted_'+ col_suffix, 
                      'delta_inc_test_'+ col_suffix,  'delta_inc_pred_'+ col_suffix,
                      'delta_case_test_'+ col_suffix, 'delta_case_pred_'+ col_suffix,
                      'error_y_'+ col_suffix, 'error_delta_inc_'+ col_suffix, 'error_delta_case_'+ col_suffix]
        
        
        counties_xgb_21 = counties_xgb_21.merge(testing_df[test_cols], how='left', on='GEOID')
        
        time_end = time.time()
        
        print('Training Model {} in time step {} finished in {} seconds!'.format(model, i, round(time_end-time_start, 2)))
    

In [ ]:
# save the output for performance analysis
counties_xgb_21.to_file(output_dir + 'STXGB_3week_pred.geojson', driver='GeoJSON')

## 6. Training STXGB for three-week (21-day) prediction horizon

In [ ]:
counties_xgb_28 = counties_shp.copy()

In [ ]:
train_r2_xgb_28 = dict()
train_rmse_xgb_28 = dict()
train_mae_xgb_28 = dict()
test_rmse_xgb_28 = dict()
test_mae_xgb_28 = dict()
tuned_params_xgb_28 = dict()



models=['base', 'safegraph', 'facebook', 'safegraph_full']
features = [base_features, safegraph_features, facebook_features, safegraph_full_features]


xgb_params = dict(learning_rate=np.arange(0.05,0.3,0.05), 
                     n_estimators=np.arange(100,1000,100), 
                     gamma = np.arange(1,10,1),
                     subsample = np.arange(0.1,0.5,0.05),
                     max_depth=[int(i) for i in np.arange(1,10,1)]) 



for i in range(time_steps):
    time_start = time.time()
    
    training_df = covid_df.iloc[:(i+training_size)*num_counties,:]
    testing_df = covid_df.iloc[(i+training_size)*num_counties:(i+training_size+testing_size)*num_counties,:]
    
    
    for model,feature in zip(models, features):
        
        # in the 4-week prediction model, the target variable is LOG_DELTA_INC_RATE_T_28
        X_train = training_df[feature]
        y_train = training_df['LOG_DELTA_INC_RATE_T_28']
        X_test = testing_df[feature]
        y_test = testing_df['LOG_DELTA_INC_RATE_T_28'] 

        #scaling X
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)


        #inititalization
        xgb_model = xgboost.XGBRegressor(booster='gbtree', seed=42, verbosity=0) 
        
        #cross validation
        xgb_cv = RandomizedSearchCV(xgb_model, xgb_params, random_state=28, 
                                    scoring='neg_root_mean_squared_error', n_jobs=-1)
        xgb_optimized = xgb_cv.fit(X_train, y_train)
        best_xgb = xgb_optimized.best_estimator_
        tuned_params_xgb_28[model, i] = xgb_optimized.best_params_

        # model evaluation for training set
        r2_train_xgb = round(best_xgb.score(X_train, y_train),2)
        train_r2_xgb_28[model, i] = r2_train_xgb

        y_train_predicted_xgb = best_xgb.predict(X_train)
        rmse_train_xgb = (np.sqrt(mean_squared_error(y_train, y_train_predicted_xgb)))
        train_rmse_xgb_28[model, i] = rmse_train_xgb
        train_mae_xgb_28[model, i] =  mean_absolute_error(y_train, y_train_predicted_xgb)

        # model evaluation for test set
        y_test_predicted_xgb = best_xgb.predict(X_test)
        rmse_test_xgb = (np.sqrt(mean_squared_error(y_test, y_test_predicted_xgb)))
        test_rmse_xgb_28[model, i] = rmse_test_xgb
        test_mae_xgb_28[model, i] = mean_absolute_error(y_test, y_test_predicted_xgb)
        
        #feature importance
        
        xgb_importance = pd.concat([pd.DataFrame(feature, columns={'variable'}),
                                    pd.DataFrame(np.transpose(best_xgb.feature_importances_), columns={'Importance'})],
                                   axis = 1) 
        xgb_importance.sort_values(by='Importance', ascending=False, inplace=True)
        xgb_importance.to_csv(output_dir + 'XGB_importance_28_' + model + '_' + str(i) + '.csv')
        
        # # add labels and predictions to a county data frame
        col_suffix = model +'_' + str(i)
        
        testing_df.loc[:,'y_test_'+ col_suffix] = y_test
        testing_df.loc[:,'y_predicted_'+ col_suffix] = y_test_predicted_xgb
        
        testing_df['delta_inc_test_'+ col_suffix] = np.exp(testing_df['y_test_'+ col_suffix]) - 1
        testing_df['delta_inc_pred_'+ col_suffix] = np.exp(testing_df['y_predicted_'+ col_suffix]) - 1
        
        testing_df['delta_case_test_'+ col_suffix] = (testing_df['delta_inc_test_'+ col_suffix] * 
                                                      testing_df['POPULATION']) / 10000
        
        testing_df['delta_case_pred_'+ col_suffix] = (testing_df['delta_inc_pred_'+ col_suffix] * 
                                                      testing_df['POPULATION']) / 10000
        
        testing_df['error_y_'+ col_suffix] = testing_df['y_test_'+ col_suffix] - testing_df['y_predicted_'+ col_suffix]
        
        testing_df['error_delta_inc_'+ col_suffix] = testing_df['delta_inc_test_'+ col_suffix] - \
                                                        testing_df['delta_inc_pred_'+ col_suffix]
        
        testing_df['error_delta_case_'+ col_suffix] = testing_df['delta_case_test_'+ col_suffix] - \
                                                        testing_df['delta_case_pred_'+ col_suffix]
        
        test_cols = ['GEOID',  
                      'y_test_'+ col_suffix, 'y_predicted_'+ col_suffix, 
                      'delta_inc_test_'+ col_suffix,  'delta_inc_pred_'+ col_suffix,
                      'delta_case_test_'+ col_suffix, 'delta_case_pred_'+ col_suffix,
                      'error_y_'+ col_suffix, 'error_delta_inc_'+ col_suffix, 'error_delta_case_'+ col_suffix]
        
        
        counties_xgb_28 = counties_xgb_28.merge(testing_df[test_cols], how='left', on='GEOID')
        
        time_end = time.time()
        
        print('Training Model {} in time step {} finished in {} seconds!'.format(model, i, round(time_end-time_start, 2)))
    

In [ ]:
# save the output for performance analysis
counties_xgb_28.to_file(output_dir + 'STXGB_4week_pred.geojson', driver='GeoJSON')

## 7. Saving GeoJson files for error analysis

In [ ]:
# creating error maps and csvs for mapping and plotting STXGB models


xgb_error_shapefiles = [counties_xgb, counties_xgb_14, counties_xgb_21, counties_xgb_28]
xgb_error_names = ['XGB_defalt', 'XGB_14', 'XGB_21', 'XGB_28']



for error_shp, error_name in zip(xgb_error_shapefiles, xgb_error_names):
    
    models=['base', 'safegraph', 'facebook', 'safegraph_full']
    
    
    for model in models:

        model_cols = [col for col in y_error if model in col]
        xgb_errors['error_y_' + model + '_avg'] = xgb_errors[model_cols].mean(axis=1)


    for model in models:

        model_cols = [col for col in delta_inc_error if model in col]
        xgb_errors['error_delta_inc_' + model + '_avg'] = xgb_errors[model_cols].mean(axis=1)

    for model in models:

        model_cols = [col for col in delta_case_error if model in col]
        xgb_errors['error_delta_case_' + model + '_avg'] = xgb_errors[model_cols].mean(axis=1)

    xgb_errors.to_file(output_dir + 'error_map_' + error_name + '.geojson', driver='GeoJSON')
    
    #Save Maps
    error_df_xgb = pd.DataFrame(index=index_temp, columns=['RMSE case', 'MAE case', 'RMSE inc', 'MAE inc'], dtype=float)


    for model in models:
        for i in range(time_steps):
            rmse_case = np.sqrt(mean_squared_error(xgb_errors['delta_case_test_' + model + '_' + str(i)],
                                               xgb_errors['delta_case_pred_' + model + '_' + str(i)]))
            rmse_inc = np.sqrt(mean_squared_error(xgb_errors['delta_inc_test_' + model + '_' + str(i)],
                                               xgb_errors['delta_inc_pred_' + model + '_' + str(i)]))

            mae_case = mean_absolute_error(xgb_errors['delta_case_test_' + model + '_' + str(i)],
                                               xgb_errors['delta_case_pred_' + model + '_' + str(i)])
            mae_inc = mean_absolute_error(xgb_errors['delta_inc_test_' + model + '_' + str(i)],
                                               xgb_errors['delta_inc_pred_' + model + '_' + str(i)])

            error_df_xgb.loc[model, i] = [rmse_case, mae_case, rmse_inc, mae_inc]
            
    error_df_xgb.to_csv(output_dir + 'error_' + error_name + '.csv')
